# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have collected data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Sit', 'Left', 'Stand', 'Frog', 'Right', 'Crouch', 'Belly', 'StandInit', 'Knee', 'HeadBack', 'Back']


In [78]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    
    # YOUR CODE HERE
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    data = np.asarray(data)
    target = [i] * len(data)
    target = np.asarray(target)
    return data, target

In [79]:
data,target = load_pose_data(1)
target.shape

(20,)

In [80]:
# load all the data
# all_data = []
# all_target = []

# YOUR CODE HERE
all_data, all_target = load_pose_data(0)

for i, value in enumerate(classes):
    if not i == 0:
        print(i, value)
        data, target = load_pose_data(i)
        all_data = np.append(all_data, data, axis=0)
        all_target = np.append(all_target, target, axis=0)

print('total number of data', len(all_data))
all_target

1 Left
2 Stand
3 Frog
4 Right
5 Crouch
6 Belly
7 StandInit
8 Knee
9 HeadBack
10 Back
total number of data 222


array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10])

In [76]:
# permutation = np.random.permutation(len(all_data))
# n_training_data = int(len(all_data) * 0.7)
# n_training_data

155

In [103]:
# shuffle data
# permutation = np.random.permutation(len(all_data))
# n_training_data = int(len(all_data) * 0.7)
# training_data = permutation[:n_training_data]

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

all_data, all_target = unison_shuffled_copies(all_data, all_target)
# all_target = all_target.reshape(-1, 1)
all_target = ravel(all_target)
all_target.shape

(222,)

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [101]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [104]:
# YOUR CODE HERE
rest = int(len(all_data) / 5)
clf.fit(all_data[:-rest], all_target[:-rest])

SVC(C=100.0, gamma=0.001)

### predicting

In [110]:
clf.predict(all_data[-1:],), all_target[-1:]
# clf.predict(1,2)

(array([8]), array([8]))

In [111]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [112]:
# expected = []
# predicted = []
# YOUR CODE HERE
expected = all_target[:-rest]
predicted = clf.predict(all_data[:-rest],)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       1.00      1.00      1.00        18
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00         8
           4       1.00      0.90      0.95        10
           5       0.96      1.00      0.98        25
           6       1.00      1.00      1.00        18
           7       1.00      1.00      1.00        41
           8       1.00      1.00      1.00         5
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00        19

    accuracy                           0.99       178
   macro avg       1.00      0.99      0.99       178
weighted avg       0.99      0.99      0.99       178


Confusion matrix:
[[21  0  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  0  0  0  0  0]
 [ 0  0  0  8  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [113]:
# expected = []
# predicted = []

# YOUR CODE HERE
expected = all_target[-rest:]
predicted = clf.predict(all_data[-rest:],)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00        11
           8       1.00      1.00      1.00         5
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00         4

    accuracy                           1.00        44
   macro avg       1.00      1.00      1.00        44
weighted avg       1.00      1.00      1.00        44


Confusion matrix:
[[ 5  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0  4  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [68]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [71]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

(array([10]), 10)